In [2]:
import os
import boto3
import json
import requests

import sys
sys.path.append(os.getcwd())

# Required information

In [3]:
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Load environment variables from .env
aws_account_id = os.getenv('AWS_ACCOUNT_ID')

aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')
aws_region = os.getenv('AWS_REGION')

s3_bucket_name = os.getenv('S3_BUCKET_NAME')
s3_object_key = os.getenv('S3_OBJECT_KEY')

## Specify the role name and trust policy for the Lambda service
iam_role_name = os.getenv('IAM_ROLE_NAME')

## User-dependent solution variables

In [4]:
# User setup

# Platform
ecr_image_name = "serverless-example"

# Function description
func_description='SKLearn predict Lambda function'

# API
endpoint = "predict"
method_verb='POST'
stage = "test"

## Protocols and performance

In [5]:
# AWS setup
from deploy_utils.misc import load_JSON
import os

cwd=os.getcwd()

## Specify the path to the JSON file
## NOTE: Update with the actual path to your JSON file
trust_policy_file_path = cwd+"/"+"trust_policy.json"
trust_policy = load_JSON(trust_policy_file_path)

## Rate limits: Harsh since this will be public facing
## Quota: Low daily limits for the same reason
usage_constraints_file_path = cwd+"/"+"lambda_usage_constraints.json"
usage_constraints = load_JSON(usage_constraints_file_path)

## AWS Lambda execution role 
lambda_policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'


Loaded JSON:
{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}
Loaded JSON:
{'rate_limits': {'burstLimit': 10, 'rateLimit': 10.0}, 'quota': {'limit': 100, 'period': 'DAY'}}


In [6]:
usage_constraints

{'rate_limits': {'burstLimit': 10, 'rateLimit': 10.0},
 'quota': {'limit': 100, 'period': 'DAY'}}

# Clients

In [7]:
# Set up the IAM client
iam_client = boto3.client('iam', region_name=aws_region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=aws_region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=aws_region)

# Development steps

- IAM role image handling;
- ECR image
- Lambda function creation;
- API Gateway

## IAM Role Image Handling

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 



In [8]:
from deploy_utils.iam_utils import try_attach_role_policy

# The id "role_arn" will be used on lambda deployment
role_arn = try_attach_role_policy(iam_client, iam_role_name, lambda_policy_arn, trust_policy)

role_arn

'arn:aws:iam::060004687794:role/lambda-exec-role'

## ECR image

Run 2 cells below with key stroke _Shift+Enter_ to upload the docker image to ECR.

In [9]:
from deploy_utils.ecr_utils import pipe_push_image

pipe_push_image(aws_account_id, aws_region, ecr_image_name)

Logining on ECR account...
Deleting existent ECR image...
Creating ECR image...
Building docker image...
Tagging docker image...
Pushing docker image to ECR...
Docker image upload finished successfully!


# Lambda function creation

Run 2 cells below with key stroke _Shift+Enter_ to generate a Lambda Function based on Docker Image

### Create function

In [10]:
from deploy_utils.ecr_utils import build_ecr_url
from deploy_utils.lambda_utils import delete_function, create_function

# Function name (not public facing)
lambda_function_name = f'lambda-fn-{ecr_image_name}'

# Retrieve (if already exists) or create a new Lambda function
routed_url = build_ecr_url(aws_account_id, aws_region, ecr_image_name)
deletion_response = delete_function(lambda_client, lambda_function_name)
create_response = create_function(lambda_client, lambda_function_name, func_description, routed_url, role_arn)

create_response

{'ResponseMetadata': {'RequestId': '7288a327-71ff-4826-860a-11316ca0672f',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 18 Sep 2023 14:13:50 GMT',
   'content-type': 'application/json',
   'content-length': '1110',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7288a327-71ff-4826-860a-11316ca0672f'},
  'RetryAttempts': 0},
 'FunctionName': 'lambda-fn-serverless-example',
 'FunctionArn': 'arn:aws:lambda:us-east-1:060004687794:function:lambda-fn-serverless-example',
 'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
 'CodeSize': 0,
 'Description': 'SKLearn predict Lambda function',
 'Timeout': 10,
 'MemorySize': 256,
 'LastModified': '2023-09-18T14:13:49.391+0000',
 'CodeSha256': '18db8a7ef972d222a111b2d92c137f416e51a06c1deadf4d07de6b144fbb62a7',
 'Version': '4',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '22b4ca2c-2295-43f0-9b6d-e3061e7c0a34',
 'State': 'Pending',
 'StateReason': 'The function is being created.',
 'StateReasonCode': 'Creating'

### Get function

In [21]:
from deploy_utils.lambda_utils import get_function

get_response = get_function(lambda_client, lambda_function_name)

get_response

{'ResponseMetadata': {'RequestId': 'c8207a7d-d755-41ab-8d1a-0cd35a3220aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 18 Sep 2023 14:14:12 GMT',
   'content-type': 'application/json',
   'content-length': '1427',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c8207a7d-d755-41ab-8d1a-0cd35a3220aa'},
  'RetryAttempts': 0},
 'Configuration': {'FunctionName': 'lambda-fn-serverless-example',
  'FunctionArn': 'arn:aws:lambda:us-east-1:060004687794:function:lambda-fn-serverless-example',
  'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
  'CodeSize': 0,
  'Description': 'SKLearn predict Lambda function',
  'Timeout': 10,
  'MemorySize': 256,
  'LastModified': '2023-09-18T14:13:49.391+0000',
  'CodeSha256': '18db8a7ef972d222a111b2d92c137f416e51a06c1deadf4d07de6b144fbb62a7',
  'Version': '$LATEST',
  'TracingConfig': {'Mode': 'PassThrough'},
  'RevisionId': '75d8e0e6-da2c-411f-a8e3-bb25540a2f68',
  'State': 'Active',
  'LastUpdateStatus': 'Successful',
  'Packag

### Test lambda function

In [35]:
from json import dumps, loads

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=dumps({"body": example})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=lambda_function_name,
    InvocationType='RequestResponse',
    Payload=payload
)

# Get the response from the Lambda function
result = loads(response['Payload'].read())

print(result)

{'statusCode': 200, 'headers': {'Content-Type': 'application/json'}, 'body': '[1, 4, 9, 16, 25]', 'isBase64Encoded': False}


## API Gateway setup

Run the cells below to set the Lambda Function as an Endpoint on API Gateway.

In [ ]:
from deploy_utils.api_gateway_utils import delete_apis_by_name, has_api

delete_apis_by_name(gateway_client, rest_api_name)

In [26]:
from deploy_utils.api_gateway_utils import deploy_rest_api, build_api_url

# Define the name of the API (not public facing)
rest_api_name = lambda_function_name + '-api'

deployment_reponse = deploy_rest_api(\
    gateway_client, lambda_client, \
    aws_account_id, aws_region, \
    lambda_function_name, rest_api_name, \
    endpoint, method_verb, \
    usage_constraints, stage \
)

deployment_reponse

{'url': 'https://vxr3w2o3g0.execute-api.us-east-1.amazonaws.com/test/predict/',
 'api_key': 'yQ7qztmZVE2VAd6Qa4BeV30uxxEXsrN56LJV2x0j',
 'usage_plan_id': '3mitkv',
 'rest_api_id': 'vxr3w2o3g0'}

In [27]:
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, aws_region, endpoint, stage)

print(api_url)


https://vxr3w2o3g0.execute-api.us-east-1.amazonaws.com/test/predict/


In [28]:
api_key=deployment_reponse['api_key']
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, aws_region, endpoint, stage)

print(api_key)
print(api_url)


yQ7qztmZVE2VAd6Qa4BeV30uxxEXsrN56LJV2x0j
https://vxr3w2o3g0.execute-api.us-east-1.amazonaws.com/test/predict/


### Test API Gateway endpoint

In [34]:
# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=json.dumps({"body": example})

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)
resp.json()


[1, 4, 9, 16, 25]